In [2]:
import re
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def semshift2wic(semshift_raw, set_prefix, mode='median'):
    df = pd.read_csv(semshift_raw, sep='\t')
    data, data_labels = [], []

    def extract_spans(string):
        regex = re.compile(r'(?P<start><b><i>)(?P<word>\w+)(?P<nonalpha>\W*)(?P<end><\/i><\/b>)')
        spans = []
        while True:
            match = regex.search(string)
            if match is None:
                break
            start = len(string[:match.start('start')])
            end = match.end('word') - len('<b><i>')
            string = \
                string[:match.start('start')] + \
                string[match.start('word'):match.end('nonalpha')] + \
                string[match.end('end'):]
            spans.append((start, end))
        return spans, string

    ex_id = 0
    for row_id, row in enumerate(df.itertuples()):
        lemma = row.word
        pos = 'NOUN'
        sent1_spans, sent1 = extract_spans(row.sent1)
        sent2_spans, sent2 = extract_spans(row.sent2)
        for (s1, e1) in sent1_spans:
            for (s2, e2) in sent2_spans:
                idx = f'{set_prefix}.{ex_id}'
                ex_id += 1
                example = {
                    'id': idx,
                    'lemma': lemma,
                    'pos': pos,
                    'sentence1': sent1,
                    'sentence2': sent2,
                    'start1': s1,
                    'end1': e1,
                    'start2': s2,
                    'end2': e2,
                    'grp': row.group
                }
                if mode == 'median':
                    score = sorted([row.annotator1, row.annotator2, row.annotator3, row.annotator4, row.annotator5])[2]
                elif mode == 'mean':
                    score = row.mean

                if score >= 3.0:
                    tag = 'T'
                elif score <= 2.0:
                    tag = 'F'
                else:
                    continue

                label = {
                    'id': idx,
                    'tag': tag,
                    'row': row_id,
                    'score': score
                }
                data.append(example)
                data_labels.append(label)

    return data, data_labels

In [3]:
data, data_labels = semshift2wic('data/RuSemShift/rusemshift_1_raw_annotations.tsv', 'dev.rusemshift_1')

In [15]:
with open('data/RuSemShift/dev.rusemshift_1.data', 'w', encoding='utf-8') as data_f:
    json.dump(data, data_f, indent=4, ensure_ascii=False)
with open('data/RuSemShift/dev.rusemshift_1.gold', 'w', encoding='utf-8') as labels_f:
    json.dump(data_labels, labels_f, indent=4, ensure_ascii=False)

In [16]:
data, data_labels = semshift2wic('data/RuSemShift/rusemshift_2_raw_annotations.tsv', 'dev.rusemshift_2')

In [17]:
with open('data/RuSemShift/dev.rusemshift_2.data', 'w', encoding='utf-8') as data_f:
    json.dump(data, data_f, indent=4, ensure_ascii=False)
with open('data/RuSemShift/dev.rusemshift_2.gold', 'w', encoding='utf-8') as labels_f:
    json.dump(data_labels, labels_f, indent=4, ensure_ascii=False)

In [9]:
lens = []

for i, sample in enumerate(data):
    lens.append(len(sample['sentence1'].split()))
    lens.append(len(sample['sentence2'].split()))
    
    if i == 6741 // 2:
        print(i)
        print(sample)
        print(sample['sentence2'][sample['start2']:sample['end2']])
        print(len(sample['sentence2'].split()))
    
#     if (lens[-1] == 2842 or lens[-2] == 2842):
#         print(sample)
#         print(i)
    
print(max(lens))

3370
{'id': 'dev.rusemshift_1.3370', 'lemma': 'стиль', 'pos': 'NOUN', 'sentence1': 'Индеец хотел поразить Ихтиандра и повел его по главным улицам города -- Авенида Альвар, Вертис, показал площадь Виктории с кафедральным собором и ратушей в мавританском стиле, площадь Фуэрто и площадь Двадцать пятого мая [25 мая 1810 года провинции Ла-Плата образовали революционный союз -- "Хунту", арестовали местную власть, провозгласили временное правительство и отделились от Испании.] с обелиском Свободы, окруженным прекрасными деревьями, президентский дворец.', 'sentence2': 'Попытаемся представить себе фотографа в соломенном сомбреро, в сношенных сандалиях на больших пыльных ступнях загорелых ног; август, набережная, толпа, фоном -- фонтан и корпуса санатория, построенного в большом стиле конца тридцатых годов; группа здешних отдыхающих сбилась в кучу, и молодцу в сомбреро не сразу удается обуздать это пугливое и бестолковое стадо; но вот наконец мало-мальски пристойный порядок достигнут, отдыхающие

In [24]:
1776 / (len(data) * 2)

0.25701881331403764

In [31]:
with open('data/eval_12_True_True/1-2_2.data') as f:
    train_data = json.load(f)
    
len(train_data)

100

In [32]:
train_data[60]

{'id': 'test.scd_2.60',
 'lemma': 'апостол',
 'pos': 'NOUN',
 'sentence1': 'Существовали легенды о видениях воскресшего Лазаря, апостола Павла и множество других, из которых самыми значительными и распространенными в ХII веке были три загробные видения ирландского происхождения: "Путешествие св.',
 'sentence2': 'Дай мне обстоятельства, при которых я мог бы вырасти в полную меру возраста Христова, как говорит апостол Павел (Еф. 4,13).',
 'start1': 52,
 'end1': 60,
 'start2': 98,
 'end2': 105,
 'grp': 'COMPARE'}